# Сбор данных

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import os

# Прочтение датасеты с результатами матчей
db = sqlite3.connect('football_stats.db')
conn = sqlite3.connect('football_stats.db')
df = pd.read_sql("SELECT DISTINCT * FROM football_stats ORDER BY start_datetime", conn)

conn.commit()
conn.close()

In [2]:
df

,region,tournament,stage,start_date,start_time,start_datetime,home,guest,goal_home,goal_guest,...,save_home,save_guest,foul_home,foul_guest,yellow_home,yellow_guest,xG_home,xG_guest,red_home,red_guest
0,ЕВРОПА,ЛИГА ЧЕМПИОНОВ,1/2,25.06.2019,16:00,2019-06-25 16:00:00,Тре Пенне (Сан),Санта-Колома (Анд),0.0,1.0,...,4.0,2.0,8.0,3.0,0.0,0.0,NaN,NaN,0.0,0.0
1,ЕВРОПА,ЛИГА ЧЕМПИОНОВ,1/2,25.06.2019,21:45,2019-06-25 21:45:00,Фероникели (Кос),Линкольн Ред Импс (Гиб),1.0,0.0,...,1.0,2.0,7.0,10.0,4.0,4.0,NaN,NaN,0.0,0.0
2,ЕВРОПА,ЛИГА ЕВРОПЫ,ФИНАЛ,27.06.2019,20:30,2019-06-27 20:30:00,Барри (Уэл),Клифтонвилл (Сир),0.0,0.0,...,2.0,0.0,10.0,6.0,1.0,1.0,NaN,NaN,0.0,0.0
3,ЕВРОПА,ЛИГА ЕВРОПЫ,ФИНАЛ,27.06.2019,20:30,2019-06-27 20:30:00,Прогрес Нидеркорн (Люк),Кардифф Метрополитан (Уэл),1.0,0.0,...,0.0,5.0,12.0,5.0,1.0,1.0,NaN,NaN,0.0,0.0
4,ЕВРОПА,ЛИГА ЕВРОПЫ,ФИНАЛ,27.06.2019,21:00,2019-06-27 21:00:00,Клаксвик (Фар),Тре Фиори (Сан),5.0,1.0,...,3.0,5.0,15.0,11.0,4.0,3.0,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14464,ЕВРОПА,CONFERENCE LEAGUE,ТУР 3,07.11.2024,23:00,2024-11-07 23:00:00,ЛАСК,Серкль Брюгге,0.0,0.0,...,2.0,2.0,10.0,7.0,1.0,4.0,1.16,0.19,0.0,1.0
14465,ЕВРОПА,CONFERENCE LEAGUE,ТУР 3,07.11.2024,23:00,2024-11-07 23:00:00,Хартс,Хайденхайм,0.0,2.0,...,1.0,5.0,8.0,13.0,0.0,1.0,1.46,0.67,0.0,0.0
14466,ЕВРОПА,CONFERENCE LEAGUE,ТУР 3,07.11.2024,23:00,2024-11-07 23:00:00,Челси,Ноа,8.0,0.0,...,7.0,10.0,9.0,5.0,0.0,2.0,5.80,0.61,0.0,0.0
14467,ЕВРОПА,CONFERENCE LEAGUE,ТУР 3,07.11.2024,23:00,2024-11-07 23:00:00,Юргорден,Панатинаикос,2.0,1.0,...,0.0,4.0,14.0,12.0,3.0,4.0,1.66,0.49,0.0,0.0


In [3]:
df.columns

Index(['region', 'tournament', 'stage', 'start_date', 'start_time',
       'start_datetime', 'home', 'guest', 'goal_home', 'goal_guest',
       'possession_home', 'possession_guest', 'shot_home', 'shot_guest',
       'shot_on_target_home', 'shot_on_target_guest', 'shot_miss_home',
       'shot_miss_guest', 'free_kick_home', 'free_kick_guest', 'corner_home',
       'corner_guest', 'offside_home', 'offside_guest', 'save_home',
       'save_guest', 'foul_home', 'foul_guest', 'yellow_home', 'yellow_guest',
       'xG_home', 'xG_guest', 'red_home', 'red_guest'],
      dtype='object')

# Метод анализа фолов команды

Количество фолов, совершённых команд традиционно слабо зависит от того, против кого они играют и в каком чемпионате. Кроме того, количество фолов очень конкретно показывают степень грубости игры команды. Поэтому фолы будут рассматриваться не в контексте матчей, а по конкретним командам. 

In [4]:
def fouls_stat(df, team, start_period = '2000-01-01'):
    
    # отдельный датасет с играми по каждой анализируемой команде
    df = df[((df['home'] == team) | (df['guest'] == team)) & (df['start_datetime'] > start_period)]

    # Сбор значения фолов
    def get_fouls(row):
        if row['home'] == team:
            return row['foul_home']  
        elif row['guest'] == team:
            return row['foul_guest']
    
    fouls = df.apply(get_fouls, axis=1)
    minimum = (fouls > fouls.rolling(window=3).min().shift())[3:].value_counts()
    
    try: 
        prob_minimum = round((minimum[True] / minimum.sum()) * 100, 2)
        last_minimum = fouls.rolling(window=3).min().shift().values[-1]
        mean_minimum = fouls.rolling(window=3).min().shift().mean()
    except:
        prob_minimum = 0
        last_minimum = 0
        mean_minimum = 0
    
    # Расчет количества и доли верных прогнозов для ТБ фолов 8.5
    counter_085 = (fouls > 8.5).value_counts()
    try: prob_085 = round((counter_085[True] / counter_085.sum()) * 100, 2)
    except: prob_085 = 0

    # Расчет количества и доли верных прогнозов для ТБ фолов 9.5
    counter_095 = (fouls > 9.5).value_counts()
    try: prob_095 = round((counter_095[True] / counter_095.sum()) * 100, 2)
    except: prob_095 = 0

    # Расчет количества и доли верных прогнозов для ТБ фолов 10.5
    counter_105 = (fouls > 10.5).value_counts()
    try: prob_105 = round((counter_105[True] / counter_105.sum()) * 100, 2)
    except: prob_105 = 0

    # Расчет количества и доли верных прогнозов для ТБ фолов 11.5
    counter_115 = (fouls > 11.5).value_counts()
    try: prob_115 = round((counter_115[True] / counter_115.sum()) * 100, 2)
    except: prob_115 = 0

    # Расчет количества и доли верных прогнозов для ТБ фолов 12.5
    counter_125 = (fouls > 12.5).value_counts()
    try: prob_125 = round((counter_125[True] / counter_125.sum()) * 100, 2)
    except: prob_125 = 0

    # Расчет количества и доли верных прогнозов для ТБ фолов 13.5
    counter_135 = (fouls > 13.5).value_counts()
    try: prob_135 = round((counter_135[True] / counter_135.sum()) * 100, 2)
    except: prob_135 = 0

    # Расчет количества и доли верных прогнозов для ТБ фолов 14.5
    counter_145 = (fouls > 14.5).value_counts()
    try: prob_145 = round((counter_145[True] / counter_145.sum()) * 100, 2)
    except: prob_145 = 0

    # Расчет количества и доли верных прогнозов для ТБ фолов 14.5
    counter_155 = (fouls > 15.5).value_counts()
    try: prob_155 = round((counter_155[True] / counter_155.sum()) * 100, 2)
    except: prob_155 = 0

    return prob_minimum, last_minimum, mean_minimum, prob_085, prob_095, prob_105, prob_115, prob_125, prob_135, prob_145, prob_155

def fouls_prob_df(df, teams, start_period = '2000-01-01'):

    probs_minimum = []
    lasts_minimum =  []
    means_minimum = []
    probs_085 = [] 
    probs_095 = []
    probs_105 = []
    probs_115 = []
    probs_125 = []
    probs_135 = []
    probs_145 = []
    probs_155 = []

    for team in teams:

        prob_minimum, last_minimum, mean_minimum, prob_085, prob_095, prob_105, prob_115, prob_125, prob_135, prob_145, prob_155 = fouls_stat(df, team, start_period = start_period)
    
        # Добавляем нужные статистики по каждой команде за последние N матчей
        probs_minimum.append(prob_minimum)
        lasts_minimum.append(last_minimum)
        means_minimum.append(mean_minimum)
        probs_085.append(prob_085)
        probs_095.append(prob_095)
        probs_105.append(prob_105)
        probs_115.append(prob_115)
        probs_125.append(prob_125)
        probs_135.append(prob_135)
        probs_145.append(prob_145)
        probs_155.append(prob_155)
    
    # Датасет по всем командам и всем статистикам
    res = pd.DataFrame(
                {'team': teams,
                 'prob_minimum': probs_minimum,
                 'last_minimum': lasts_minimum,
                 'mean_minimum': means_minimum,
                 'prob_085': probs_085,
                 'prob_095': probs_095,
                 'prob_105': probs_105,
                 'prob_115': probs_115,
                 'prob_125': probs_125,
                 'prob_135': probs_135,
                 'prob_145': probs_145,
                 'prob_155': probs_155}
              ).sort_values(by=['prob_minimum'], ascending=False).reset_index(drop=True).dropna()

    return res

In [5]:
# Команды из топ-5 лиг + РПЛ этого сезона
teams = df[(df['start_datetime'] > '2024-06-15') & (df['region'] != 'ЕВРОПА')]['home'].unique()

In [6]:
res_curr = fouls_prob_df(df, teams, start_period = '2024-06-15')
res_curr.head(50)

,team,prob_minimum,last_minimum,mean_minimum,prob_085,prob_095,prob_105,prob_115,prob_125,prob_135,prob_145,prob_155
0,Санкт-Паули,100.00,7.0,6.400000,75.00,62.50,37.50,25.00,12.50,12.50,12.50,12.50
1,Брайтон,100.00,8.0,7.500000,66.67,55.56,55.56,44.44,33.33,33.33,11.11,11.11
2,Вердер,100.00,7.0,5.200000,75.00,50.00,25.00,12.50,12.50,12.50,12.50,12.50
3,Акрон Тольятти,90.91,5.0,7.454545,71.43,64.29,50.00,50.00,50.00,42.86,21.43,21.43
4,Манчестер Сити,90.00,5.0,4.700000,46.15,23.08,7.69,7.69,0.00,0.00,0.00,0.00
5,Айнтрахт Ф,88.89,6.0,5.444444,58.33,58.33,25.00,25.00,16.67,0.00,0.00,0.00
6,Вильярреал,87.50,11.0,9.625000,72.73,72.73,63.64,45.45,36.36,27.27,18.18,9.09
7,Венеция,87.50,9.0,6.750000,81.82,63.64,63.64,54.55,36.36,27.27,27.27,18.18
8,Хайденхайм,87.50,12.0,8.375000,90.91,81.82,72.73,72.73,54.55,36.36,18.18,9.09
9,Вест Хэм,85.71,6.0,9.857143,90.00,80.00,70.00,60.00,50.00,50.00,50.00,40.00


In [7]:
res_curr[(res_curr['team'] == 'Хетафе')]

,team,prob_minimum,last_minimum,mean_minimum,prob_085,prob_095,prob_105,prob_115,prob_125,prob_135,prob_145,prob_155
27,Хетафе,77.78,17.0,14.888889,100.0,100.0,100.0,91.67,83.33,83.33,75.0,58.33


In [8]:
res_curr[(res_curr['team'] == 'Спартак Москва')]

,team,prob_minimum,last_minimum,mean_minimum,prob_085,prob_095,prob_105,prob_115,prob_125,prob_135,prob_145,prob_155
42,Спартак Москва,72.73,17.0,12.181818,85.71,85.71,78.57,71.43,71.43,71.43,64.29,64.29


In [9]:
res_curr[(res_curr['team'] == 'Эспаньол')]

,team,prob_minimum,last_minimum,mean_minimum,prob_085,prob_095,prob_105,prob_115,prob_125,prob_135,prob_145,prob_155
70,Эспаньол,66.67,14.0,9.0,83.33,75.0,75.0,75.0,66.67,66.67,25.0,25.0


In [10]:
res_curr[(res_curr['team'] == 'Атлетик')]

,team,prob_minimum,last_minimum,mean_minimum,prob_085,prob_095,prob_105,prob_115,prob_125,prob_135,prob_145,prob_155
99,Атлетик,53.85,9.0,10.076923,87.5,81.25,81.25,75.0,62.5,50.0,43.75,25.0


In [11]:
res_curr[(res_curr['team'] == 'Рубин')]

,team,prob_minimum,last_minimum,mean_minimum,prob_085,prob_095,prob_105,prob_115,prob_125,prob_135,prob_145,prob_155
79,Рубин,63.64,13.0,10.727273,100.0,78.57,78.57,71.43,64.29,57.14,42.86,42.86


In [13]:
pd.concat([res_curr[(res_curr['team'] == 'Хетафе')],
           res_curr[(res_curr['team'] == 'Спартак Москва')],
           res_curr[(res_curr['team'] == 'Эспаньол')],
           res_curr[(res_curr['team'] == 'Атлетик')],
           res_curr[(res_curr['team'] == 'Рубин')]], ignore_index=True)

,team,prob_minimum,last_minimum,mean_minimum,prob_085,prob_095,prob_105,prob_115,prob_125,prob_135,prob_145,prob_155
0,Хетафе,77.78,17.0,14.888889,100.00,100.00,100.00,91.67,83.33,83.33,75.00,58.33
1,Спартак Москва,72.73,17.0,12.181818,85.71,85.71,78.57,71.43,71.43,71.43,64.29,64.29
2,Эспаньол,66.67,14.0,9.000000,83.33,75.00,75.00,75.00,66.67,66.67,25.00,25.00
3,Атлетик,53.85,9.0,10.076923,87.50,81.25,81.25,75.00,62.50,50.00,43.75,25.00
4,Рубин,63.64,13.0,10.727273,100.00,78.57,78.57,71.43,64.29,57.14,42.86,42.86


In [14]:
res_curr[res_curr['last_minimum'] > 11].sort_values(by='last_minimum', ascending=False)

,team,prob_minimum,last_minimum,mean_minimum,prob_085,prob_095,prob_105,prob_115,prob_125,prob_135,prob_145,prob_155
27,Хетафе,77.78,17.0,14.888889,100.00,100.00,100.00,91.67,83.33,83.33,75.00,58.33
42,Спартак Москва,72.73,17.0,12.181818,85.71,85.71,78.57,71.43,71.43,71.43,64.29,64.29
48,Ланс,71.43,16.0,12.571429,90.00,90.00,90.00,90.00,80.00,80.00,70.00,70.00
66,Боруссия М,66.67,15.0,10.833333,100.00,100.00,77.78,66.67,44.44,44.44,33.33,11.11
80,Факел,63.64,14.0,11.363636,92.86,92.86,85.71,78.57,71.43,64.29,57.14,50.00
70,Эспаньол,66.67,14.0,9.000000,83.33,75.00,75.00,75.00,66.67,66.67,25.00,25.00
62,Жирона,69.23,14.0,9.923077,87.50,87.50,62.50,50.00,43.75,37.50,18.75,18.75
61,Челси,70.00,13.0,10.100000,100.00,69.23,61.54,53.85,38.46,15.38,7.69,7.69
72,Алавес,66.67,13.0,11.777778,100.00,100.00,91.67,83.33,83.33,58.33,41.67,33.33
68,Вальядолид,66.67,13.0,12.333333,100.00,100.00,100.00,100.00,75.00,58.33,50.00,16.67


**Вывод:** 1/5 - очень плохой результат. Нужно придумывать другую модель.

# Метод прогнозирования фолов в матче

Далее попробуем прогнозировать количество фолов в матче, а не отдельно по командам.

In [15]:
def make_fouls_stats(df, teams, regions, matches_start, matches_end=None):
   
    # Здесь сохранятся данные (средняя, медиана, минимум, максимум, 20% квантиль, 80% квантиль)
    means = []
    medians = []
    mins = []
    maxes = []
    quantiles_20 = []
    quantiles_80 = []
    
    # Проход по всем командам, которые есть в переменной teams
    for team in teams:
        
        # Сбор значения из фолов команды
        def get_fouls(row):
            if row['home'] == team:
                return row['foul_home']  
            elif row['guest'] == team:
                return row['foul_guest']

        # отдельный датасет с играми по каждой анализируемой команде
        df_team = df[(df['home'] == team) | (df['guest'] == team)]
    
        # Если у команды меньше или ровно 3 игр, то не считаем статистику, так как мало данных
        if df_team.shape[0] <= 5:
            mins.append(None)
            medians.append(None)
            means.append(None)
            quantiles_20.append(None)
            quantiles_80.append(None)
            maxes.append(None)
            continue

        # Считаем фолы команды за последние N матчей (обычно N=5)
        fouls = df_team.apply(get_fouls, axis=1)[matches_start:matches_end]

        # Добавляем нужные статистики по каждой команде за последние N матчей
        means.append(round(fouls.mean(), 2))
        medians.append(round(fouls.median(), 2))
        quantiles_20.append(round(fouls.quantile(q=0.2), 2))
        quantiles_80.append(round(fouls.quantile(q=0.8), 2))
        mins.append(round(fouls.min(), 2))
        maxes.append(round(fouls.max(), 2))

    # Датасет по всем командам и всем статистикам
    res = pd.DataFrame(
                {'Команда': teams,
                 'Страна': regions,
                 'Минимум': mins,
                 '20% квантиль': quantiles_20,
                 'Медиана': medians,
                 'Среднее': means,
                 '80% квантиль': quantiles_80,
                 'Максимум': maxes}
              ).sort_values(by=['Среднее'], ascending=False).reset_index(drop=True).dropna()

    # Делаем синтетические матч по всем командам, в которых статистики по фолам по синтетическим матч - это сумма значений по каждой команде в матче 
    df_merged = res.merge(res, how='cross')
    df_merged = df_merged[(df_merged['Команда_x'] != df_merged['Команда_y']) & (df_merged['Страна_x'] == df_merged['Страна_y'])]
    df_merged['Минимум'] = df_merged['Минимум_x'] + df_merged['Минимум_y']
    df_merged['20% квантиль'] = df_merged['20% квантиль_x'] + df_merged['20% квантиль_y']
    df_merged['Медиана'] = df_merged['Медиана_x'] + df_merged['Медиана_y']
    df_merged['Среднее'] = df_merged['Среднее_x'] + df_merged['Среднее_y']
    df_merged['80% квантиль'] = df_merged['80% квантиль_x'] + df_merged['80% квантиль_y']
    df_merged['Максимум'] = df_merged['Максимум_x'] + df_merged['Максимум_y']
    df_merged = df_merged[['Команда_x', 'Команда_y', 'Страна_x', 'Минимум', '20% квантиль', 'Медиана', 'Среднее', '80% квантиль', 'Максимум']]

    df_merged = df_merged.rename(columns={'Команда_x': 'home', 'Команда_y': 'guest', 'Страна_x': 'Страна'})
    
    return df_merged

In [16]:
teams_reg = df[(df['start_datetime'] > '2024-06-15') & (df['region'] != 'ЕВРОПА')][['home', 'region']].drop_duplicates()
teams = teams_reg['home']
regions = teams_reg['region']

In [17]:
df_6 = df[df['region'] != 'ЕВРОПА']
df_6

,region,tournament,stage,start_date,start_time,start_datetime,home,guest,goal_home,goal_guest,...,save_home,save_guest,foul_home,foul_guest,yellow_home,yellow_guest,xG_home,xG_guest,red_home,red_guest
80,РОССИЯ,ПРЕМЬЕР-ЛИГА,ТУР 1,12.07.2019,20:00,2019-07-12 20:00:00,Арсенал Тула,Динамо Москва,1.0,1.0,...,5.0,1.0,14.0,16.0,0.0,3.0,NaN,NaN,0.0,0.0
81,РОССИЯ,ПРЕМЬЕР-ЛИГА,ТУР 1,13.07.2019,16:30,2019-07-13 16:30:00,Урал,Уфа,3.0,2.0,...,3.0,3.0,11.0,9.0,0.0,1.0,NaN,NaN,0.0,0.0
82,РОССИЯ,ПРЕМЬЕР-ЛИГА,ТУР 1,13.07.2019,19:00,2019-07-13 19:00:00,Спартак Москва,Сочи,1.0,0.0,...,4.0,4.0,13.0,10.0,4.0,2.0,NaN,NaN,0.0,0.0
83,РОССИЯ,ПРЕМЬЕР-ЛИГА,ТУР 1,13.07.2019,21:30,2019-07-13 21:30:00,Ростов,Оренбург,2.0,1.0,...,2.0,4.0,15.0,15.0,2.0,3.0,NaN,NaN,0.0,0.0
84,РОССИЯ,ПРЕМЬЕР-ЛИГА,ТУР 1,14.07.2019,16:30,2019-07-14 16:30:00,Крылья Советов,ЦСКА,2.0,0.0,...,2.0,4.0,13.0,12.0,1.0,3.0,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14410,ИТАЛИЯ,СЕРИЯ А,ТУР 11,04.11.2024,20:30,2024-11-04 20:30:00,Эмполи,Комо,1.0,0.0,...,2.0,3.0,17.0,9.0,1.0,0.0,0.61,0.37,0.0,0.0
14411,ИТАЛИЯ,СЕРИЯ А,ТУР 11,04.11.2024,22:45,2024-11-04 22:45:00,Лацио,Кальяри,2.0,1.0,...,0.0,3.0,12.0,17.0,3.0,8.0,2.65,0.24,0.0,2.0
14412,АНГЛИЯ,ПРЕМЬЕР-ЛИГА,ТУР 10,04.11.2024,23:00,2024-11-04 23:00:00,Фулхэм,Брентфорд,2.0,1.0,...,1.0,10.0,3.0,10.0,2.0,1.0,1.37,0.59,0.0,0.0
14413,ИСПАНИЯ,ПРИМЕРА,ТУР 12,04.11.2024,23:00,2024-11-04 23:00:00,Сельта,Хетафе,1.0,0.0,...,2.0,2.0,12.0,11.0,1.0,4.0,1.92,0.54,0.0,1.0


In [18]:
fouls_res = make_fouls_stats(df_6, teams, regions, -10)
fouls_res

,home,guest,Страна,Минимум,20% квантиль,Медиана,Среднее,80% квантиль,Максимум
1,Спартак Москва,Факел,РОССИЯ,18.0,30.2,38.0,35.6,42.2,46.0
5,Спартак Москва,Рубин,РОССИЯ,17.0,25.4,32.5,33.6,41.8,53.0
14,Спартак Москва,Локомотив Москва,РОССИЯ,17.0,27.4,31.0,32.2,37.0,52.0
19,Спартак Москва,Динамо Махачкала,РОССИЯ,16.0,27.0,33.0,32.1,38.2,45.0
26,Спартак Москва,Ростов,РОССИЯ,11.0,26.2,32.0,31.6,39.8,46.0
...,...,...,...,...,...,...,...,...,...
12510,Манчестер Сити,Фулхэм,АНГЛИЯ,6.0,11.8,18.5,17.8,24.0,26.0
12516,Манчестер Сити,Брайтон,АНГЛИЯ,10.0,11.8,17.0,17.5,23.0,28.0
12517,Манчестер Сити,Кристал Пэлас,АНГЛИЯ,10.0,12.4,17.5,17.4,22.4,26.0
12518,Манчестер Сити,Ноттингем Форест,АНГЛИЯ,9.0,12.4,17.5,17.4,22.4,28.0


# Проверка точности модели

Сделаем проверку для матчей чемпионатов, которые проводились с 01.11.2024 по 03.11.2024.

In [19]:
df_train = df[(df['start_datetime'] < '2024-11-01') & (df['region'] != 'ЕВРОПА')]
df_train

,region,tournament,stage,start_date,start_time,start_datetime,home,guest,goal_home,goal_guest,...,save_home,save_guest,foul_home,foul_guest,yellow_home,yellow_guest,xG_home,xG_guest,red_home,red_guest
80,РОССИЯ,ПРЕМЬЕР-ЛИГА,ТУР 1,12.07.2019,20:00,2019-07-12 20:00:00,Арсенал Тула,Динамо Москва,1.0,1.0,...,5.0,1.0,14.0,16.0,0.0,3.0,NaN,NaN,0.0,0.0
81,РОССИЯ,ПРЕМЬЕР-ЛИГА,ТУР 1,13.07.2019,16:30,2019-07-13 16:30:00,Урал,Уфа,3.0,2.0,...,3.0,3.0,11.0,9.0,0.0,1.0,NaN,NaN,0.0,0.0
82,РОССИЯ,ПРЕМЬЕР-ЛИГА,ТУР 1,13.07.2019,19:00,2019-07-13 19:00:00,Спартак Москва,Сочи,1.0,0.0,...,4.0,4.0,13.0,10.0,4.0,2.0,NaN,NaN,0.0,0.0
83,РОССИЯ,ПРЕМЬЕР-ЛИГА,ТУР 1,13.07.2019,21:30,2019-07-13 21:30:00,Ростов,Оренбург,2.0,1.0,...,2.0,4.0,15.0,15.0,2.0,3.0,NaN,NaN,0.0,0.0
84,РОССИЯ,ПРЕМЬЕР-ЛИГА,ТУР 1,14.07.2019,16:30,2019-07-14 16:30:00,Крылья Советов,ЦСКА,2.0,0.0,...,2.0,4.0,13.0,12.0,1.0,3.0,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14355,ИТАЛИЯ,СЕРИЯ А,ТУР 10,30.10.2024,22:45,2024-10-30 22:45:00,Аталанта,Монца,2.0,0.0,...,3.0,4.0,14.0,11.0,2.0,3.0,1.15,0.11,0.0,0.0
14356,ИТАЛИЯ,СЕРИЯ А,ТУР 10,30.10.2024,22:45,2024-10-30 22:45:00,Ювентус,Парма,2.0,2.0,...,3.0,4.0,7.0,5.0,2.0,0.0,2.26,1.13,0.0,0.0
14357,ИТАЛИЯ,СЕРИЯ А,ТУР 10,31.10.2024,20:30,2024-10-31 20:30:00,Дженоа,Фиорентина,0.0,1.0,...,3.0,4.0,13.0,10.0,4.0,2.0,0.78,0.46,0.0,0.0
14358,ИТАЛИЯ,СЕРИЯ А,ТУР 10,31.10.2024,22:45,2024-10-31 22:45:00,Комо,Лацио,1.0,5.0,...,2.0,1.0,14.0,18.0,2.0,5.0,0.99,3.27,1.0,1.0


In [20]:
fouls_train = make_fouls_stats(df_train, teams, regions, -10)
fouls_train

,home,guest,Страна,Минимум,20% квантиль,Медиана,Среднее,80% квантиль,Максимум
2,Спартак Москва,Факел,РОССИЯ,14.0,28.2,38.0,34.4,42.2,46.0
3,Спартак Москва,Рубин,РОССИЯ,17.0,26.4,34.5,34.2,41.8,53.0
19,Спартак Москва,Локомотив Москва,РОССИЯ,17.0,26.6,31.0,31.8,37.0,52.0
23,Спартак Москва,Динамо Махачкала,РОССИЯ,16.0,26.4,32.5,31.5,37.4,45.0
25,Спартак Москва,Пари НН,РОССИЯ,14.0,27.2,33.0,31.4,37.4,45.0
...,...,...,...,...,...,...,...,...,...
12518,Манчестер Сити,Эвертон,АНГЛИЯ,9.0,11.8,19.5,17.6,21.4,26.0
12519,Манчестер Сити,Челси,АНГЛИЯ,8.0,12.8,19.5,17.6,22.2,24.0
12520,Манчестер Сити,Брайтон,АНГЛИЯ,10.0,11.8,18.5,17.5,22.4,25.0
12522,Манчестер Сити,Манчестер Юнайтед,АНГЛИЯ,7.0,10.8,19.5,17.4,22.4,27.0


In [21]:
df_test = df[(df['start_datetime'] > '2024-11-01') & (df['start_datetime'] < '2024-11-05') & (df['region'] != 'ЕВРОПА')]
df_test['foul_total'] = df_test['foul_guest'] + df_test['foul_home']
df_test = df_test[['home', 'guest', 'foul_total']]
df_test

C:\Users\Asus\AppData\Local\Temp\ipykernel_42356\89462316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['foul_total'] = df_test['foul_guest'] + df_test['foul_home']


,home,guest,foul_total
14360,Динамо Москва,Ростов,27.0
14361,Монако,Анже,28.0
14362,Байер,Штутгарт,27.0
14363,Алавес,Мальорка,31.0
14364,Лилль,Лион,22.0
14365,Ньюкасл Юнайтед,Арсенал,34.0
14366,Осасуна,Вальядолид,33.0
14367,Болонья,Лечче,23.0
14368,Айнтрахт Ф,Бохум,21.0
14369,Бавария,Унион Берлин,11.0


In [22]:
# Средняя
df_final = pd.merge(fouls_train, df_test, on=['home', 'guest'], how='inner')

# Расчет вероятности
counter_min = (df_final['foul_total'] > df_final['Минимум']).value_counts()
prob_min = round((counter_min[True] / counter_min.sum()) * 100, 2)
counter_20 = (df_final['foul_total'] > df_final['20% квантиль']).value_counts()
prob_20 = round((counter_20[True] / counter_20.sum()) * 100, 2)
counter_mean = (df_final['foul_total'] > df_final['Среднее']).value_counts()
prob_mean = round((counter_mean[True] / counter_mean.sum()) * 100, 2)
counter_median = (df_final['foul_total'] > df_final['Медиана']).value_counts()
prob_median = round((counter_median[True] / counter_median.sum()) * 100, 2)
counter_80 = (df_final['foul_total'] > df_final['80% квантиль']).value_counts()
prob_80 = round((counter_80[True] / counter_80.sum()) * 100, 2)
counter_max = (df_final['foul_total'] > df_final['Максимум']).value_counts()
prob_max = round((counter_max[True] / counter_max.sum()) * 100, 2)

print(f'Минимум: {prob_min}%')
print(f'20% квантиль: {prob_20}%')
print(f'Медиана: {prob_median}%')
print(f'Среднее: {prob_mean}%')
print(f'80% квантиль: {prob_80}%')
print(f'Максиму: {prob_max}%')

df_final

Минимум: 94.44%
20% квантиль: 79.63%
Медиана: 53.7%
Среднее: 53.7%
80% квантиль: 25.93%
Максиму: 5.56%


,home,guest,Страна,Минимум,20% квантиль,Медиана,Среднее,80% квантиль,Максимум,foul_total
0,Факел,Химки,РОССИЯ,13.0,21.0,28.5,26.00,31.2,34.0,31.0
1,Эмполи,Комо,ИТАЛИЯ,20.0,21.8,27.5,27.70,31.8,38.0,26.0
2,Болонья,Лечче,ИТАЛИЯ,18.0,20.0,26.0,25.70,29.4,34.0,23.0
3,Сент-Этьен,Страсбур,ФРАНЦИЯ,17.0,21.4,25.0,27.00,30.4,46.0,13.0
4,Алавес,Мальорка,ИСПАНИЯ,15.0,21.4,24.5,26.20,30.8,43.0,31.0
5,Монца,Милан,ИТАЛИЯ,9.0,15.2,24.0,23.90,32.4,42.0,33.0
6,Удинезе,Ювентус,ИТАЛИЯ,18.0,22.4,24.0,25.40,29.6,34.0,26.0
7,Вулверхэмптон,Кристал Пэлас,АНГЛИЯ,17.0,18.6,24.5,24.80,29.6,34.0,16.0
8,Брест,Ницца,ФРАНЦИЯ,10.0,17.8,24.0,23.90,30.8,36.0,13.0
9,Локомотив Москва,Рубин,РОССИЯ,18.0,21.4,27.5,29.80,34.8,55.0,20.0


**Вывод:** 20% и 80% квантили дают правильную нижнюю и верхнюю границу оценки диапазона фолов примерно в 75%-80% случаях, что можно считать отличным результатом. Далее проведём проверку на всех исторических данных, то есть применим эту модель для всех матчей последних 5 сезонов.

# Backtesting

In [52]:
# Расчет оценок границ фолов в матче
def match_forecast(df, teams, regions):
    
    forecast = make_fouls_stats(df, teams, regions, -5).iloc[0, :]
    
    forecast_min = forecast['20% квантиль']
    forecast_max = forecast['80% квантиль']

    return forecast_min, forecast_max

# Объединение прогнозов с реальным результатом матча
def match_analysis(df, game):
    
    teams = game[['home', 'guest']].values
    region = game['region']
    datetime = game['start_datetime']
    fouls_total = game['foul_home'] + game['foul_guest']

    forecast_min, forecast_max = match_forecast(df[df['start_datetime'] < datetime], teams, 2 * region)

    return teams[0], teams[1], region, forecast_min, forecast_max, fouls_total

res = []

for i in range(200, df_6.shape[0]):
    
    game = df_6.iloc[i, :]
    
    try:
        res.append(match_analysis(df_6, game))
    except: 
        continue

res_df = pd.DataFrame(res, columns=['home', 'guest', 'region', 'forecast_min', 'forecast_max', 'fouls_total'])

# Расчёт вероятностей по команде
team_res = []

for team in teams:

    team_df = res_df[(res_df['home'] == team) | (res_df['guest'] == team)]

    region = team_df['region'].values[0]

    min_right = round((team_df['fouls_total'] > team_df['forecast_min']).mean() * 100, 2)
    max_right = round((team_df['fouls_total'] < team_df['forecast_max']).mean() * 100, 2)
    all_right = round(((team_df['fouls_total'] > team_df['forecast_min']) & ((team_df['fouls_total'] < team_df['forecast_max']))).mean() * 100, 2)

    team_res.append([team, region, min_right, max_right, all_right])

team_res_df = pd.DataFrame(team_res, columns=['team', 'region', 'min_right_prob', 'max_right_prob', 'all_right_prob'])

# Создаем синтетические матчи и по ним считаем считаем вероятность удачного прогноза
matches_res_df = team_res_df.merge(team_res_df, how='cross')
matches_res_df = matches_res_df[(matches_res_df['team_x'] != matches_res_df['team_y']) & (matches_res_df['region_x'] == matches_res_df['region_y'])]
matches_res_df['min_right_prob'] = np.min((matches_res_df['min_right_prob_x'], matches_res_df['min_right_prob_y']), axis=0)
matches_res_df['max_right_prob'] = np.min((matches_res_df['max_right_prob_x'], matches_res_df['max_right_prob_y']), axis=0)
matches_res_df['all_right_prob'] = np.min((matches_res_df['all_right_prob_x'], matches_res_df['all_right_prob_y']), axis=0)
matches_res_df = matches_res_df[['team_x', 'team_y', 'region_x', 'min_right_prob', 'max_right_prob', 'all_right_prob']]

# Добавим прогноз на следующий матч
forecast_min = []
forecast_max = []

for i in matches_res_df.values:
    forecasts = match_forecast(df_6, [i[0], i[1]], 2 * [i[2]])
    forecast_min.append(forecasts[0])
    forecast_max.append(forecasts[1])

matches_res_df['forecast_min'] = forecast_min
matches_res_df['forecast_max'] = forecast_max

matches_res_df = matches_res_df.rename(columns={'team_x': 'home', 'team_y': 'guest', 'region_x': 'region'})

In [56]:
matches_res_df[(matches_res_df['region'] == 'РОССИЯ')
               & (matches_res_df['home'] == 'Локомотив Москва')
               & (matches_res_df['guest'] == 'Факел')]

,home,guest,region,min_right_prob,max_right_prob,all_right_prob,forecast_min,forecast_max
9,Локомотив Москва,Факел,РОССИЯ,77.27,68.18,46.97,25.8,35.2


In [76]:
matches_res_df[(matches_res_df['region'] == 'РОССИЯ')
               & (matches_res_df['home'] == 'Спартак Москва')
               & (matches_res_df['guest'] == 'Акрон Тольятти')]

,home,guest,region,min_right_prob,max_right_prob,all_right_prob,forecast_min,forecast_max
1691,Спартак Москва,Акрон Тольятти,РОССИЯ,70.0,72.08,55.84,25.4,37.0


In [75]:
matches_res_df[(matches_res_df['region'] == 'ИСПАНИЯ')
               & (matches_res_df['home'] == 'Вильярреал')
               & (matches_res_df['guest'] == 'Алавес')]

,home,guest,region,min_right_prob,max_right_prob,all_right_prob,forecast_min,forecast_max
6141,Вильярреал,Алавес,ИСПАНИЯ,75.16,77.07,52.23,26.4,35.6


In [85]:
matches_res_df[(matches_res_df['region'] == 'ГЕРМАНИЯ')
               & (matches_res_df['home'] == 'Санкт-Паули')
               & (matches_res_df['guest'] == 'Бавария')]

,home,guest,region,min_right_prob,max_right_prob,all_right_prob,forecast_min,forecast_max
10076,Санкт-Паули,Бавария,ГЕРМАНИЯ,65.09,88.17,53.25,13.8,20.2


In [106]:
pd.concat([matches_res_df[(matches_res_df['home'] == 'Локомотив Москва') & (matches_res_df['guest'] == 'Факел')],
           matches_res_df[(matches_res_df['home'] == 'Спартак Москва') & (matches_res_df['guest'] == 'Акрон Тольятти')],
           matches_res_df[(matches_res_df['home'] == 'Вильярреал') & (matches_res_df['guest'] == 'Алавес')],
           matches_res_df[(matches_res_df['home'] == 'Санкт-Паули') & (matches_res_df['guest'] == 'Бавария')]], ignore_index=True)

,home,guest,region,min_right_prob,max_right_prob,all_right_prob,forecast_min,forecast_max
0,Локомотив Москва,Факел,РОССИЯ,77.27,68.18,46.97,25.8,35.2
1,Спартак Москва,Акрон Тольятти,РОССИЯ,70.00,72.08,55.84,25.4,37.0
2,Вильярреал,Алавес,ИСПАНИЯ,75.16,77.07,52.23,26.4,35.6
3,Санкт-Паули,Бавария,ГЕРМАНИЯ,65.09,88.17,53.25,13.8,20.2


**Вывод:** 3/4 матчей зашли, что так же считается отличным результатом для прогнозирующей модели. 